In [1]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast
from bs4 import BeautifulSoup
import acquire
import json

### Connect to API and scrape the data needed
#### get the access token and 200 response 

In [ ]:
def get_api():   
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *;'
    r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [2]:
access_token = acquire.connect_api()

In [ ]:
access_token

In [3]:
tf = acquire.connect_to_games()

In [4]:
tf

<Response [200]>

In [5]:
wrapper = acquire.run_wrapper()

In [ ]:
wrapper

In [ ]:
df = acquire.get_game_library(wrapper)
df.head()

In [ ]:
from igdb.wrapper import IGDBWrapper
wrapper = IGDBWrapper(f'{Client_ID}', f'Bearer {access_token}')

In [ ]:
games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {0 * 409};')

In [ ]:
url = 'https://api.igdb.com/v4/age_ratings'
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'category,checksum,content_descriptions,rating,rating_cover_url,synopsis;'
response = requests.post(url, headers=headers, data = data)
print(response.text)

In [ ]:
testgames = []
for i in range(0, 409):
    games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 409};')
    testgames.append(games)

In [ ]:
len(testgames)

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
test = soup.select('div', class_= 'toc-wrapper')
test

## scraping endpoints
### scrape the end points of each category 

In [ ]:
access_token


### platform_families

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/platform_families' ,data=data ,  headers=headers)

platform_families = wrapper.api_request('platform_families', 'fields *;limit 500;')

In [ ]:
x = json.loads(platform_families)

platform_families =pd.DataFrame(x)

In [ ]:
platform_families.head()

In [ ]:
platform_families.shape

In [6]:
def get_platform_families():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_families' ,data=data ,  headers=headers)
    platform_families = wrapper.api_request('platform_families', 'fields *;limit 500;')
    x = json.loads(platform_families)
    platform_families =pd.DataFrame(x)
    return platform_families

In [7]:
platform_families = get_platform_families()
platform_families.head()

,id,name,slug,checksum
0,5,Nintendo,nintendo,398e5c87-f7db-6265-6da6-433441385208
1,4,Linux,linux,be8b89fa-823c-790a-1d5a-7e74315c1b27
2,2,Xbox,xbox,5bb9bcec-4aa7-45b8-129b-a6f88932942e
3,3,Sega,sega,bd1a9cc4-3632-33b0-ff4b-feab8cd15001
4,1,PlayStation,playstation,eadc0208-7972-9364-88b9-d2a9de75a998


### platform_version_release_dates

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/platform_version_release_dates' ,data=data ,  headers=headers)

platform_version_release_dates = wrapper.api_request('platform_version_release_dates', 'fields *;limit 500;')

In [ ]:
x = json.loads(platform_version_release_dates)

platform_version_release_dates =pd.DataFrame(x)

In [ ]:
platform_version_release_dates.head()

In [ ]:
platform_version_release_dates.shape

In [8]:
def get_platform_version_release():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_release_dates' ,data=data ,  headers=headers)
    platform_version_release_dates = wrapper.api_request('platform_version_release_dates', 'fields *;limit 500;')
    x = json.loads(platform_version_release_dates)
    platform_version_release_dates =pd.DataFrame(x)
    return platform_version_release_dates

In [9]:
platform_version_release_dates = get_platform_version_release()
platform_version_release_dates

,id,category,date,human,m,region,y,checksum,updated_at
0,555,2,567907200,1987,12,8,1987,186c07ec-0d93-dc0f-6c4a-5fdb5a6138fa,NaN
1,338,0,1189555200,"Sep 12, 2007",9,3,2007,15e5b0aa-5bb6-ef30-ee85-0ef94cb3581f,NaN
2,334,0,1125532800,"Sep 01, 2005",9,7,2005,a4c921d5-1894-9a40-d20f-05959cd69a38,NaN
3,331,0,1415923200,"Nov 14, 2014",11,3,2014,3ebb8dc8-b8fa-f11a-b729-228b9dac35c4,NaN
4,333,0,1389830400,"Jan 16, 2014",1,7,2014,a15ecea5-cb71-916f-5bec-66ad7f571b19,NaN
...,...,...,...,...,...,...,...,...,...
391,660,0,1547942400,"Jan 20, 2019",1,7,2019,29210c4c-26c6-212c-e0ec-c0d6bdd23be9,1.660316e+09
392,659,0,1413763200,"Oct 20, 2014",10,2,2014,ddac84c3-8f5a-8372-fee3-5e05cc9517f5,1.660316e+09
393,661,2,725760000,1992,12,8,1992,d6cb796b-0679-d99b-30d8-decdf573187d,1.660320e+09
394,662,0,945043200,"Dec 13, 1999",12,5,1999,6fa54af0-85f8-088f-294c-d95dbbbb2a70,1.660322e+09


### platform_version_companies

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/platform_version_companies' ,data=data ,  headers=headers)

platform_version_companies = wrapper.api_request('platform_version_companies', 'fields *;limit 500;')

In [ ]:
x = json.loads(platform_version_companies)

platform_version_companies =pd.DataFrame(x)
platform_version_companies.head()

In [ ]:
platform_version_companies.shape

In [11]:
def get_platform_version_companies():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_companies' ,data=data ,  headers=headers)
    platform_version_companies = wrapper.api_request('platform_version_companies', 'fields *;limit 500;')
    x = json.loads(platform_version_companies)
    platform_version_companies =pd.DataFrame(x)
    return platform_version_companies

In [12]:
platform_version_companies=get_platform_version_companies()
platform_version_companies.head()

,id,company,developer,manufacturer,checksum,comment
0,286,11986,True,True,e5035f31-713a-1036-2968-dd353b74db3b,NaN
1,283,10922,False,True,6cd3e57c-2757-cc07-94a6-35ff7085e4c9,NaN
2,282,14429,False,True,172b6f24-a49c-c304-6fb3-78f6bf120a19,NaN
3,281,12385,True,False,d6051460-e7dc-4e77-80ca-e5df378cd304,NaN
4,280,19831,True,True,2f509e1b-75d2-36db-4129-4bf89d50343b,NaN


### platforms

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/platforms' ,data=data ,  headers=headers)

platforms = wrapper.api_request('platforms', 'fields *;limit 500;')

In [ ]:
x = json.loads(platforms)

platforms =pd.DataFrame(x)
platforms.head()

In [ ]:
platforms

In [13]:
def get_platforms():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platforms' ,data=data ,  headers=headers)
    platforms = wrapper.api_request('platforms', 'fields *;limit 500;')
    x = json.loads(platforms)
    platforms =pd.DataFrame(x)
    return platforms

In [14]:
platforms = get_platforms()
platforms.shape

(195, 16)

### release_dates

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/release_dates' ,data=data ,  headers=headers)

release_dates = wrapper.api_request('release_dates', 'fields *;limit 500;')

In [ ]:
x = json.loads(release_dates)

release_dates =pd.DataFrame(x)
release_dates.head()

In [ ]:
release_dates.shape

In [16]:
def get_release_dates():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/release_dates' ,data=data ,  headers=headers)
    release_dates = wrapper.api_request('release_dates', 'fields *;limit 500;')
    x = json.loads(release_dates)
    release_dates =pd.DataFrame(x)
    return release_dates

In [17]:
release_dates = get_release_dates()
release_dates.shape

(500, 12)

### platform_versions

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/platform_versions' ,data=data ,  headers=headers)

platform_versions = wrapper.api_request('platform_versions', 'fields *;limit 500;')

In [ ]:
x = json.loads(platform_versions)

platform_versions=pd.DataFrame(x)
platform_versions.head()

In [18]:
def get_platform_versions():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_versions' ,data=data ,  headers=headers)
    platform_versions = wrapper.api_request('platform_versions', 'fields *;limit 500;')
    x = json.loads(platform_versions)
    platform_versions=pd.DataFrame(x)
    return platform_versions

In [19]:
platform_versions = get_platform_versions()
platform_versions.shape

(312, 21)

### player_perspectives

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/player_perspectives' ,data=data ,  headers=headers)

player_perspectives = wrapper.api_request('player_perspectives', 'fields *;limit 500;')

In [ ]:
x = json.loads(player_perspectives)

player_perspectives=pd.DataFrame(x)
player_perspectives.head()

In [20]:
def get_player_perspectives():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/player_perspectives' ,data=data ,  headers=headers)
    player_perspectives = wrapper.api_request('player_perspectives', 'fields *;limit 500;')
    x = json.loads(player_perspectives)
    player_perspectives=pd.DataFrame(x)
    return player_perspectives

In [21]:
player_perspectives = get_player_perspectives()
player_perspectives.shape

(7, 7)

### themes

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/themes' ,data=data ,  headers=headers)

themes = wrapper.api_request('themes', 'fields *;limit 500;')

In [ ]:
x = json.loads(themes)

themes=pd.DataFrame(x)
themes.head()

In [22]:
def get_themes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/themes' ,data=data ,  headers=headers)
    themes = wrapper.api_request('themes', 'fields *;limit 500;')
    x = json.loads(themes)
    themes=pd.DataFrame(x)
    return themes

In [23]:
themes = get_themes()
themes.shape

(22, 7)

### Read the CSV for games 

In [ ]:
df = pd.read_csv('games.csv', index_col='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
df.release_dates.isnull().sum()

In [ ]:
df.first_release_date.isnull().sum()

In [ ]:
df.dlcs.value_counts()

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.loc[df['franchises'].notnull()]

In [ ]:
df = df.drop(columns=['websites', 'created_at', 'updated_at', 'artworks', 'cover','screenshots', 'videos', 'checksum', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'franchises', 'follows', 'aggregated_rating_count'])
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.forks.value_counts()

In [ ]:
df.rename(columns={'slug': 'lower_case_name', 'aggregated_rating': 'critic_rating'}, inplace=True)
df.head()

In [ ]:
df.rating.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.loc[df['rating'].isnull()]

In [ ]:
df.release_dates.info()

In [ ]:
df.platforms.value_counts()

### Dataframe with ratings

In [ ]:
df2 = df[df['rating'].notnull()]
df2.head()

In [ ]:
df2.rating.isnull().sum()

### Dataframe with nulls in rating column 

In [ ]:
df3 = df[df['rating'].isnull()]
df3.shape